# Assignment 3

## Part 1

The goal of this part of the assignment is to provide you with practice and experience in some basic data exploration and hypothesis testing with Python. You will work with data from the “HUE bedtime procrastination study”. A cleaned version of the data is available on Canvas (`hue_week_3.csv`), as well as another file that contains data from the post-study questionnaire that participants filled out at the end of the study (`hue_questionnaire.csv`). This file contains the following information:

| Column | Description |
-----------------------|--------------------------------------------|
| `gender`          | 1 = male, 2 = female |
| `age`           | Numeric age value | 
| `chronotype`      |    Single item (7-point scale), do you consider yourself more of a <br> morning (1) or an evening person? (7) |
| `bp_scale` | Dutch version of the Bedtime Procrastination Scale |
| `motivation` | Questions pertaining to personality traits related to procrastination. <br> Single item (7-point scale), how motivated were you to go to bed on <br> time each night? (1 = not motivated, 7 = very motivated) |
| `daytime_sleepiness` | Dutch translation of the Epworth Sleepiness Scale <br> (4-point scale from 0-3; 8 questions, values summed) |
| `self_reported_effectiveness` | Single item (7-point scale), <br> do you feel more rested since the intervention |

In this part of the assignment, you will use Python to examine the post-questionnaire data in relation to the HUE data file, visualize trends and relationships, look for correlations between factors, test for significant differences between groups and build a regression model to predict bedtime delay. In order to perform the analyses, a number of transformations on the data still need to be done.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
import datetime
import sys
from io import StringIO


pd.options.display.max_rows = 20

## Exercise 1 (20 points)
Implement the following steps in Python:

<ul>
<li>
    Read the HUE data file and the questionnaire data file into two separate pandas DataFrames.
<br></li>
<li>
    Create a new DataFrame that contains the following Series:
    
| Column | Description |
-----------------------|--------------------------------------------|
| `ID` | Participant ID |
| `group` | Participant group (1 for experimental, 0 for control) |
| `delay_nights` | The number of nights the participant delayed their bedtime (range: 0-12) |
| `delay_time` | The mean time in seconds a participant delayed their bedtime <br> (total delay in seconds, divided by the number of observations <br> measured for each individual, rounded to nearest second) |
| `sleep_time` | The mean time in seconds of a participant (rounded to nearest second) |
    
    
<br></li>
<li>
    Set the index of this new DataFrame to `ID`. Note that there should only be a single row per participant ID.    
<br></li>
<li>
    Fill this new DataFrame by transforming data from the DataFrame about participants' bedtimes (from the HUE data file).
<br></li>
<li>
    Merge this new DataFrame with the post-questionnaire data and store the resulting DataFrame in a new variable. Perform this merging operation of the two DataFrames in such a way that the resulting Data Frame only contains IDs that were present in both datasets.
<br></li>
<li>
    Remove the rows that have NaN values in this merged DataFrame.
</li>
</ul> 

In [2]:
sleepdatafile   = 'hue_week_3.csv'
surveydatafile  = 'hue_questionnaire.csv'

In [5]:
def read_data(sleepdatafile, surveydatafile):
# YOUR CODE HERE
    colnames = ['ID', 'group', 'delay_nights', 'delay_time', 'sleep_time']
    sleep = pd.read_csv(sleepdatafile, sep = ',', encoding ='utf_8')
    survey = pd.read_csv(surveydatafile, sep = ',', encoding ='utf_8')
    
    
    
# YOUR CODE ENDS HERE

In [94]:
import datetime as dt
import statistics 
def date_from_time(datetime_str):
    today = dt.date.today()
    tomorrow = today + dt.timedelta(days=1)
    time = dt.datetime.strptime(datetime_str, '%H:%M').time()
    if time < dt.time(16,0,0): #morning next date
        return dt.datetime.combine(tomorrow, time)
    else:
        return dt.datetime.combine(today, time)

def gth_(x, y):
    if is_nan((x, y)):
        return False
    return date_from_time(x) > date_from_time(y)

def diff(a):
    x, y = a[1], a[0]
    return (date_from_time(x) - date_from_time(y)).total_seconds()
    
def is_nan(x):
    return x[0] != x[0] or x[1] != x[1]

def mean(l):
    if l:
        return round(statistics.mean(l))
    return 0


sleep = pd.read_csv(sleepdatafile, sep = ',', encoding ='utf_8')
col_names = sleep.columns
rows = []
index = []
for _, row in sleep.iterrows():
    intended = [x for (y, x) in zip(col_names, row) if "Intended Bedtime" in y]
    actual = [x for (y, x) in zip(col_names, row) if "Actual Bedtime" in y]
    rise_time = [x for (y, x) in zip(col_names, row) if "Rise Time" in y]
    
    delayed_nights = list(filter(lambda x : gth_(x[1], x[0]), zip(intended, actual)))
    delay_time = mean(list(map(lambda x : diff(x), delayed_nights)))
    sleep_time = mean(list(map(diff, filter(lambda x : not is_nan(x), zip(actual, rise_time[1:])))))
  
    row_ = [row['Condition'], len(delayed_nights), delay_time, sleep_time]
    rows.append(row_)
    index.append(row['ID'])


df_ = pd.DataFrame(rows, columns=['group', 'delay_nights', 'delay_time', 'sleep_time'], index=index)    

df_ 

,group,delay_nights,delay_time,sleep_time
1,0,10,6030,31222
2,0,7,2494,27880
4,1,7,4149,30345
5,1,9,2620,32267
6,0,7,5417,29925
...,...,...,...,...
58,0,11,3622,28380
60,1,5,4188,16695
61,0,9,5707,30608
63,0,6,1090,30982


In [103]:
survey = pd.read_csv(surveydatafile, sep = ',', encoding ='utf_8', index_col=0)
survey
survey.merge(df_, left_index=True, right_index=True, suffixes=(False, False))

,gender,age,chronotype,bp_scale,motivation,daytime_sleepiness,self_reported_effectiveness,group,delay_nights,delay_time,sleep_time
1,2,20,7,6.11,4,17,4,0,10,6030,31222
2,2,25,5,5.22,4,21,2,0,7,2494,27880
4,2,30,1,6.67,5,14,5,1,7,4149,30345
5,2,27,5,2.67,6,12,6,1,9,2620,32267
6,1,27,6,5.11,6,14,3,0,7,5417,29925
...,...,...,...,...,...,...,...,...,...,...,...
53,2,43,6,4.67,6,16,5,1,0,0,0
55,1,26,4,5.00,2,15,1,0,5,3672,27012
58,1,35,7,6.33,6,12,2,0,11,3622,28380
61,2,39,7,5.89,5,19,1,0,9,5707,30608


In [6]:
mergedDfNoNan = read_data(sleepdatafile, surveydatafile)
mergedDfNoNan


ID                               int64
Condition                        int64
Day 1 Date                      object
Day 1 Intended Bedtime          object
Day 1 Actual Bedtime            object
                                ...   
Day 12 Rise Time                object
Day 12 In Bed Time (sec)       float64
Day 12 Had Rise Reason          object
Day 12 Fitness                 float64
Day 12 Adherence Importance    float64
Length: 122, dtype: object


## Exercise 2 (5 points)
Use the `scipy.stats` package and, respectively, the Pearson correlation test and the Kendall rank correlation test, to calculate the following correlation coefficients:
<br></li>
<li>
    the Pearson correlation coefficient between bedtime procrastination scale (`bp_scale`, a personality trait) and mean time spent delaying bedtime,    
<br></li>
<li>
    the Kendall rank correlation coefficient between age and mean time spent delaying bedtime,
<br></li>
<li>
    the Pearson correlation coefficient between mean time spent delaying bedtime and daytime sleepiness.
</li>
</ul> 

Save them into the variables `r1`, `tau`, `r2`.

In [ ]:
def calculate_correlations(mergedDfNoNan):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
r1, pvalue1, tau, pvalue2, r2, pvalue3 = calculate_correlations(mergedDfNoNan)

statistics = [r1,tau,r2]
pvalues = [pvalue1, pvalue2, pvalue3]

print("Correlation tests:")
for (statistic, pvalue) in zip(statistics, pvalues):
    print('The value of the test statistic is:',statistic)
    print('The p-value is:', pvalue,'\n')


## Exercise 3 (15 points)
Use the `scipy.stats` package to determine whether there are significant differences (at 5\% significance level) between the experimental group and the control group in terms of:
<br></li>
<li>
    the number of nights participants delayed their bedtime,    
<br></li>
<li>
    the time participants spent in bed each night,
<br></li>
<li>
    the mean time participants spent delaying their bedtime.
</li>
</ul> 

Use the t-test or the Wilcoxon rank-sum test to reach a conclusion and use knowledge gained in the courses Statistics and Statistical Data Analysis to determine which statistical test is appropriate. Save the conclusions - either the string 'significant difference' or 'no significant difference' - into the variables `dif1`, `dif2`, `dif3`.

\* Note that in the final assignment you are expected to explicitly motivate the choice of an appropriate test, in this exercise you do not have to.

In [ ]:
def perform_tests(mergedDf):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
dif1, dif2, dif3 = perform_tests(mergedDfNoNan) 

print('The number of nights participants delayed their bedtime:', dif1)
print('The time participants spent in bed each night:', dif2)
print('The mean time participants spent delaying their bedtime:', dif3)


## Exercise 4 (15 points)
Use `statsmodels.api` to build a regression model for `delay_time` on the predictors `age`, `chronotype` and `bp_scale`. Return the coefficients of the model, and the conclusion whether the model is significant by using the string 'significant' or 'not significant'.

\* Convince yourself that the basic diagnostics for this model are ok. Here not, but in the final assignment you are expected to explicitly check the diagnostics.

In [ ]:
def regression_analysis(mergedDfNoNan):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
parameters, significant = regression_analysis(mergedDfNoNan)

print('The parameters of the model are:')
print(parameters)
print('\nThe model is', significant)


## Exercise 5 (15 points)
Create three distinct, meaningful, well-crafted visualizations that either provide insight into the data, or help support your conclusions. This means creating three different kinds of plots (not three boxplots, or three scatterplots for example). Interpret and discuss your findings.

In [ ]:
# Plot 1
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
# Plot 2
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
# Plot 3
# YOUR CODE HERE

# YOUR CODE ENDS HERE

# Part 2
The goal of this part of the assignment is to provide you with practice in implementing MapReduce in Python. Using the `map_reduce_hue.csv` dataset, you will implement two simple MapReduce algorithms.

<a href="https://towardsdatascience.com/a-beginners-introduction-into-mapreduce-2c912bb5e6ac">First read this webpage!</a>

In the ideal situation, we would have access to multiple nodes in the cloud to test our MapReduce functions. Instead, we are going to simulate such an environment in this notebook. We are going to feed the Map function a line of the file in each call (as if this is running on a node in the cloud). The Map function will print the result of the computation to the standard output. When all Map function have processed all lines of the file, the Reduce function is going to collect the output of the Map functions (all the intermediate results that were printed). We do this line by line as well as if the output of a Map function is sent to a Reduce function directly. The Reduce function will then transform the intermediate results to obtain the final answer that one wants to compute. 

Since the Map function is using the `print()` function to communicate to the Reduce function, we use a smart trick! We store the standard output in a variable, and replace it by a variable. Whenever the Map function using the `print()` function, it is added in a string to the variable. After all Map functions are finished, we have collected all the output and change back to the original standard output. Now, the reduce function can use the variable to process the output of the Map functions. Note that since the Reduce function is fed line by line, the function might need to use global variables instead of local variable to store the information.

## Exercise 6 (15 points)
Write a MapReduce algorithm that counts and outputs the total number of times the fitness value is strictly higher than 50. The expected output is a single integer. In this case, the Map function should print relevant information related to the line that be used by the Reduce function. The Reduce function should read all these values, and print the total count. In this case, it might be necessary to have a global variable `totalCount`, which indicates the current count of the number of relevant lines.

In [ ]:
def mapper1(line):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
def reducer1(line):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
def mapreduce1(data):    
    old_stdout = sys.stdout
    mystdout = StringIO()
    sys.stdout = mystdout

    with open(data) as file:
        for index, line in enumerate(file):
            if index == 0:
                continue
            line = line.strip()
            mapper1(line)
        mapper1(',,,,,,,')
        
        sys.stdout = old_stdout
        mapper_lines = mystdout.getvalue().split("\n")
        mystdout.close()

        for index, line in enumerate(sorted(mapper_lines)):
            if index == 0:
                continue
            reducer1(line)
        reducer1('')

In [ ]:
totalCount = 0
mapreduce1('map_reduce_hue.csv')


## Exercise 7 (15 points)
Write a MapReduce algorithm that calculates the mean fitness per participant. Do not use any statistical packages to calculate the mean. The expected output is one line per participant, containing the participants ID and the mean of his or her fitness, separated by a tab ("\t"). The outputted lines do not have to be sorted. 

The Map function in this case is more complicated than in the previous case. Think about what information the Map function should give the Reduce function. In this case, it is necessary to have at least the variable `currentID` (indicating which ID you are processing) as global variable

In [ ]:
def mapper2(line):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
def reducer2(line):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
def mapreduce2(data):    
    old_stdout = sys.stdout
    mystdout = StringIO()
    sys.stdout = mystdout

    with open(data) as file:
        for index, line in enumerate(file):
            if index == 0:
                continue
            line = line.strip()
            mapper2(line)
        mapper2(',,,,,,,')
        
        sys.stdout = old_stdout
        mapper_lines = mystdout.getvalue().split("\n")
        mystdout.close()

        for index, line in enumerate(sorted(mapper_lines)):
            if index == 0:
                continue
            reducer2(line)
        reducer2('')

In [ ]:
currentID = ''
mapreduce2('map_reduce_hue.csv')
